In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import cerberus
import plotly.io as pio


p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

## Box plot for number of samples / libraries for human / mouse there are

In [2]:
datasets = get_sample_datasets(sample='mouse')

In [3]:
datasets = get_sample_datasets()

In [4]:
# human
df = pd.read_csv('swan/swan_metadata.tsv', sep='\t')
df.head()

,dataset,sample,classification,health_status
0,gm12878_1_1,gm12878,cell_line,NaN
1,gm12878_1_2,gm12878,cell_line,NaN
2,gm12878_1_3,gm12878,cell_line,NaN
3,gm12878_1_4,gm12878,cell_line,NaN
4,gm12878_3_1,gm12878,cell_line,NaN


In [5]:
# mouse
df = pd.read_csv('../mouse/lr_bulk/swan/mouse_metadata.tsv', sep='\t')
df.head()

,dataset,b6cast,tissue,age,sex,rep,5x_v_wt,disease_status,forelimb,c2c12,f1219,adrenal,sample
0,adrenal_18-20mo_m_1,True,adrenal,18-20mo,m,1.0,NaN,NaN,NaN,NaN,NaN,NaN,adrenal_18-20mo
1,gastroc_4d_m_1,True,gastroc,4d,m,1.0,NaN,NaN,NaN,NaN,NaN,NaN,gastroc_4d
2,gastroc_4d_m_2,True,gastroc,4d,m,2.0,NaN,NaN,NaN,NaN,NaN,NaN,gastroc_4d
3,heart_18-20mo_m_1,True,heart,18-20mo,m,1.0,NaN,NaN,NaN,NaN,NaN,NaN,heart_18-20mo
4,heart_18-20mo_m_2,True,heart,18-20mo,m,2.0,NaN,NaN,NaN,NaN,NaN,NaN,heart_18-20mo


In [6]:
file_ids = pd.read_csv('../mouse/lr_bulk/file_to_hr.tsv',
                       sep='\t', header=None, names=['file_id', 'hr'])
file_ids['hr'] = file_ids.hr.str.replace('-', '_')
file_ids['hr'] = file_ids.hr.str.replace('18_20', '18-20')

In [7]:
file_ids.head()

,file_id,hr
0,ENCFF684SLZ,adrenal_18-20mo_m_1
1,ENCFF643TDK,gastroc_4d_m_1
2,ENCFF821NRP,gastroc_4d_m_2
3,ENCFF800GKP,heart_18-20mo_m_1
4,ENCFF510KLD,heart_18-20mo_m_2


In [8]:
meta = pd.read_csv('../mouse/lr_bulk/metadata.tsv', sep='\t')
meta = meta[['File accession', 'Biosample type']]
meta.columns = ['file_id', 'classification']
class_map = get_tissue_cell_line_map()
meta.classification = meta.classification.map(class_map)

In [10]:
meta.classification.unique()

array(['tissue', 'cell_line'], dtype=object)

In [34]:
file_ids = file_ids.merge(meta, on='file_id')

126
126


In [38]:
print(len(df.index))
df = df.merge(file_ids, left_on='dataset', right_on='hr')
print(len(df.index))
              

126
126


In [39]:
df

,dataset,b6cast,tissue,age,sex,rep,5x_v_wt,disease_status,forelimb,c2c12,f1219,adrenal,sample,file_id,hr,classification
0,adrenal_18-20mo_m_1,True,adrenal,18-20mo,m,1.0,NaN,NaN,NaN,NaN,NaN,NaN,adrenal_18-20mo,ENCFF684SLZ,adrenal_18-20mo_m_1,tissue
1,gastroc_4d_m_1,True,gastroc,4d,m,1.0,NaN,NaN,NaN,NaN,NaN,NaN,gastroc_4d,ENCFF643TDK,gastroc_4d_m_1,tissue
2,gastroc_4d_m_2,True,gastroc,4d,m,2.0,NaN,NaN,NaN,NaN,NaN,NaN,gastroc_4d,ENCFF821NRP,gastroc_4d_m_2,tissue
3,heart_18-20mo_m_1,True,heart,18-20mo,m,1.0,NaN,NaN,NaN,NaN,NaN,NaN,heart_18-20mo,ENCFF800GKP,heart_18-20mo_m_1,tissue
4,heart_18-20mo_m_2,True,heart,18-20mo,m,2.0,NaN,NaN,NaN,NaN,NaN,NaN,heart_18-20mo,ENCFF510KLD,heart_18-20mo_m_2,tissue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,f1219_1_3,False,NaN,NaN,NaN,NaN,False,NaN,False,False,True,NaN,NaN,ENCFF697PDG,f1219_1_3,cell_line
122,adrenal_gland_2_1,False,adrenal,NaN,NaN,NaN,False,NaN,False,False,False,True,adrenal,ENCFF694ZVY,adrenal_gland_2_1,tissue
123,adrenal_gland_2_2,False,adrenal,NaN,NaN,NaN,False,NaN,False,False,False,True,adrenal,ENCFF343ZMH,adrenal_gland_2_2,tissue
124,adrenal_gland_1_1,False,adrenal,NaN,NaN,NaN,False,NaN,False,False,False,True,adrenal,ENCFF498FMF,adrenal_gland_1_1,tissue
